In [1]:
#endeener1

import pandas as pd
import numpy as np
from scipy.optimize import minimize
import scipy.io
import matplotlib.pyplot as plt

# Recommending movies(Using Collaborative filtering Algorithm)

In this assignment,I will use collaborative filtering learning algorithm and apply it to a dataset of movie ratings.This dataset consists of ratings on a scale of 1 to 5. The dataset has nu = 943 users, and nm = 1682 movies. 
The matrix Y (num movies * num users matrix) stores the ratings(from 1 to 5).
ith row of X stores feature vector of ith movie. 
Theta(j) corresponds to parameter vector related to jth user.
Obejctive is to learn both theta(parameters of a praticular user) and X(features of a particular movie) and multiply them both to predict the rating given by that user to that particular movie.
R(Its a matrix where R(i,j) = 1 if user j has given rating to movie i).


In [12]:
import os

In [13]:
os.chdir('C:/Users/jarves/Desktop/Machine Learning/Andrew NG/ex8(Anamoly detection and recommender systems)')

In [14]:
datafile = 'ex8_movies.mat'
mat = scipy.io.loadmat( datafile )
Y = mat['Y']
R = mat['R']

nm, nu = Y.shape
print(nm,nu)

1682 943


In [15]:
# Read in the movie params matrices
datafile = 'ex8_movieParams.mat'
mat = scipy.io.loadmat( datafile )
X = mat['X']
Theta = mat['Theta']
nu = int(mat['num_users'])
nm = int(mat['num_movies'])
nf = int(mat['num_features'])

# were it not for testing purpose, there's no need to load parameters X and Theta (since we'll have random initialization)
# also the nu, nm could'be been obtained from Y (just reused the nu and nm from the above)

In [16]:
#Function to flatten the parameters
def flattenParams(X, Theta):
    return np.concatenate((X.flatten(),Theta.flatten()))

# just an inverse function of flattenParams
def reshapeParams(flattened_XandTheta, nm, nu, nf):

    reX = flattened_XandTheta[:int(nm*nf)].reshape((nm,nf))
    reTheta = flattened_XandTheta[int(nm*nf):].reshape((nu,nf))
    
    return reX, reTheta

In [17]:
#Function to calculate the cost
def cofiCostFunc(params, Y, R, nf, lambd = 0.):
    
    nm, nu = Y.shape

    # retrieving X and Theta matrices from the flattened params
    X, Theta = reshapeParams(params, nm, nu, nf)
  
    error = (X.dot(Theta.T) - Y) * R
    cost = 0.5 * np.sum( np.square(error) )  # unregularized

    cost += (lambd/2.) * np.sum(np.square(X)) + (lambd/2.) * np.sum(np.square(Theta))   # regularized

    return cost

In [18]:
def cofiGrad(params, Y, R, nf, lambd = 0.):
    
    nm, nu = Y.shape
    
    # Retrieving X and Theta matrices from the flattened params
    X, Theta = reshapeParams(params, nm, nu, nf)

    error = (X.dot(Theta.T) - Y) * R
    X_grad = error.dot(Theta)
    
    # computing the gradients
    Theta_grad = error.T.dot(X)

    # Regularization stuff
    X_grad += lambd * X
    Theta_grad += lambd * Theta
    
    return flattenParams(X_grad, Theta_grad)

In [19]:
cofiCostFunc(flattenParams(X1,Theta1),Y1,R1,nf1)

22.224603725685675

In [21]:
movie_list = 'movie_ids.txt'
movies = []
with open(movie_list, encoding = "ISO-8859-1", mode = "r") as f:
    for line in f:
        movies.append(' '.join(line.strip('\n').split(' ')[1:]))
movies[1]       

'GoldenEye (1995)'

In [22]:
#User ratings
my_ratings = np.zeros((1682,1))
my_ratings[0]   = 4
my_ratings[97]  = 2
my_ratings[6]   = 3
my_ratings[11]  = 5
my_ratings[53]  = 4
my_ratings[63]  = 5
my_ratings[65]  = 3
my_ratings[68]  = 5
my_ratings[182] = 4
my_ratings[225] = 5
my_ratings[354] = 5

In [23]:
#Adding my ratings to ratings matrix
myR = my_ratings > 0
Y = np.hstack((Y,my_ratings))
R = np.hstack((R,myR))
nm, nu = Y.shape
print(Y.shape)

(1682, 944)


In [24]:
def normalizeRatings(Y, R):
    """
    Preprocess data by subtracting mean rating for every movie (every row)
    This is important because without this, a user who hasn't rated any movies
    will have a predicted score of 0 for every movie, when in reality
    they should have a predicted score of [average score of that movie].
    """

    # The mean is only counting movies that were rated
    Ymean = np.sum(Y,axis=1)/np.sum(R,axis=1)
    Ymean = Ymean.reshape((Ymean.shape[0],1))
    Ynorm= Y-Ymean
    
    return Ynorm, Ymean

In [25]:
Ynorm, Ymean = normalizeRatings(Y,R)

In [26]:
Ynorm.shape, Ymean.shape

((1682, 944), (1682, 1))

In [32]:

# Generate random initial parameters, Theta and X
X = np.random.rand(nm,nf)
Theta = np.random.rand(nu,nf)
myflat = flattenParams(X, Theta)

# Regularization parameter of 10 is used 
lambd = 10.

# Training the actual model with fmin_cg
result = scipy.optimize.fmin_cg(cofiCostFunc, x0=myflat, fprime=cofiGrad, \
                               args=(Y,R,nf,lambd), \
                                maxiter=50,disp=True,full_output=True)

         Current function value: 73014.474471
         Iterations: 50
         Function evaluations: 76
         Gradient evaluations: 76


In [33]:
print(result)
print(len(result))
print(len(result[0]))

(array([0.52503699, 0.70128771, 0.77173426, ..., 0.43162352, 0.48214664,
       0.39867701]), 73014.47447093029, 76, 76, 1)
5
26260


In [34]:
resX, resTheta = reshapeParams(result[0], nm, nu, nf)

In [35]:
prediction_matrix = resX.dot(resTheta.T)
my_predictions = prediction_matrix[:,-1] + Ymean.flatten()

In [36]:
pred_idxs_sorted = np.argsort(my_predictions)
#argsort returns the indices of the arguments in ascending order
# invert
pred_idxs_sorted[:] = pred_idxs_sorted[::-1]

print ("Top recommendations for you:")
for i in range(10):
    print ('Predicting rating %0.1f for movie %s.' %
    (my_predictions[pred_idxs_sorted[i]],movies[pred_idxs_sorted[i]]))
    
print ("\nOriginal ratings provided by user (you):")
for i in range(len(my_ratings)):
    if my_ratings[i] > 0:
        print ('Rated %d for movie %s.' % (my_ratings[i],movies[i]))

Top recommendations for you:
Predicting rating 8.4 for movie Star Wars (1977).
Predicting rating 8.4 for movie Schindler's List (1993).
Predicting rating 8.3 for movie Shawshank Redemption, The (1994).
Predicting rating 8.2 for movie Raiders of the Lost Ark (1981).
Predicting rating 8.2 for movie Titanic (1997).
Predicting rating 8.1 for movie Good Will Hunting (1997).
Predicting rating 8.1 for movie Usual Suspects, The (1995).
Predicting rating 8.0 for movie Casablanca (1942).
Predicting rating 8.0 for movie Wrong Trousers, The (1993).
Predicting rating 8.0 for movie Close Shave, A (1995).

Original ratings provided by user (you):
Rated 4 for movie Toy Story (1995).
Rated 3 for movie Twelve Monkeys (1995).
Rated 5 for movie Usual Suspects, The (1995).
Rated 4 for movie Outbreak (1995).
Rated 5 for movie Shawshank Redemption, The (1994).
Rated 3 for movie While You Were Sleeping (1995).
Rated 5 for movie Forrest Gump (1994).
Rated 2 for movie Silence of the Lambs, The (1991).
Rated 4 f